In [ ]:
import torch
from sklearn.metrics import f1_score
from graph_qa_dataset import GraphQADataset
from torch_geometric.loader import DataListLoader
from model.graph_qa_model import GAT, HGT

In [ ]:
train_dataset = GraphQADataset(split="train", data_size=20)
val_dataset = GraphQADataset(split="validation", data_size=2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = HGT(hidden_channels=64, out_channels=2, num_heads=4, num_layers=2, metadata=train_dataset.metadata)
model = model.to(device)

In [ ]:
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [ ]:
train_dataloader = DataListLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataListLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
def train(dataloader):
    model.train()
    
    total_loss = 0
    for batch in dataloader:
        data = batch[0]
        optimizer.zero_grad()
        out = model(data.x_dict, data.edge_index_dict)

        loss = loss_op(out, data["context"].y)
        print(loss)
        total_loss += loss.item()
        loss.backward()

train(train_dataloader)

In [3]:
ll = list(range(100))

In [1]:
import numpy as np

In [5]:
np.array_split(range(100), 8)

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]),
 array([26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]),
 array([39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]),
 array([52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]),
 array([64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]),
 array([76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]),
 array([88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])]